<a href="https://colab.research.google.com/github/Vinicius-A-Siqueira/CP4-IoT/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.seasonal import seasonal_decompose

# Configurações gerais
plt.rcParams.update({
    "figure.figsize": (10, 4),
    "axes.grid": True
})

DATA_FILE = "household_power_consumption.txt"

if not Path(DATA_FILE).exists():
    print(f"⚠️ Arquivo '{DATA_FILE}' não encontrado na pasta atual: {Path.cwd()}")
    print("Coloque o arquivo na mesma pasta do notebook e rode novamente.")
else:
    print("✅ Arquivo encontrado.")


In [ ]:
def load_data(filepath=DATA_FILE):
    df = pd.read_csv(
        filepath,
        sep=';',
        na_values=['?', 'NA', ''],
        low_memory=False
    )
    # Conversão de tipos numéricos
    numeric_cols = [
        'Global_active_power','Global_reactive_power','Voltage',
        'Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3'
    ]
    for c in numeric_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    return df

if Path(DATA_FILE).exists():
    df = load_data()
    print("Shape:", df.shape)
    print("Colunas:", list(df.columns))

In [ ]:
# 1- Carregue o dataset e exiba as 10 primeiras linhas.

if Path(DATA_FILE).exists():
    display(df.head(10))
else:
    print("Carregue o arquivo na pasta do notebook e rode novamente.")


In [ ]:
# 2- Diferença entre Global_active_power e Global_reactive_power

explanation = '''
- **Global_active_power**: potência **ativa** consumida (kW) — energia que realiza trabalho útil (conversão em calor, movimento, etc.).
- **Global_reactive_power**: potência **reativa** (kVAR) — energia alternada que não realiza trabalho útil diretamente; está associada a campos magnéticos/ elétricos em cargas indutivas/capacitivas e afeta a eficiência do sistema.
'''
print(explanation)

In [ ]:
# 3- Verifique valores ausentes e quantifique-os.

if Path(DATA_FILE).exists():
    missing_total = df.isna().sum()
    print("Valores ausentes por coluna:")
    print(missing_total)
    print("\nTotal de valores ausentes:", int(missing_total.sum()))
else:
    print("Arquivo não encontrado.")

In [ ]:
# 4- Converter `Date` para datetime e criar coluna com o dia da semana.

if Path(DATA_FILE).exists():
    # Converter Date
    df['Date_dt'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')
    # Dia da semana (segunda=0, domingo=6)
    df['weekday'] = df['Date_dt'].dt.day_name()  # nomes em inglês
    display(df[['Date','Date_dt','weekday']].head())
else:
    print("Arquivo não encontrado.")

In [ ]:
# 5- Filtrar registros de 2007 e calcular média de consumo diário de Global_active_power.

if Path(DATA_FILE).exists():
    df_2007 = df[df['Date_dt'].dt.year == 2007].copy()
    daily_mean_2007 = df_2007.groupby('Date_dt')['Global_active_power'].mean()
    print("Média de consumo diário (kW) em 2007:")
    display(daily_mean_2007.describe())
else:
    print("Arquivo não encontrado.")

In [ ]:
# 6- Gráfico de linha da variação de Global_active_power em um dia escolhido.

if Path(DATA_FILE).exists():
    # Combinar Date e Time em datetime
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
    # Escolha um dia com dados (ex: 2007-02-01)
    day_choice = pd.Timestamp('2007-02-01')
    mask = (df['DateTime'].dt.date == day_choice.date())
    day_df = df.loc[mask].sort_values('DateTime')
    plt.figure()
    plt.plot(day_df['DateTime'], day_df['Global_active_power'])
    plt.title(f"Global_active_power ao longo do dia — {day_choice.date()}")
    plt.xlabel("Hora")
    plt.ylabel("kW")
    plt.show()
else:
    print("Arquivo não encontrado.")

In [ ]:
# 7- Histograma de Voltage e observações sobre a distribuição.

if Path(DATA_FILE).exists():
    plt.figure()
    df['Voltage'].dropna().hist(bins=50)
    plt.title("Histograma — Voltage")
    plt.xlabel("Voltage (V)")
    plt.ylabel("Frequência")
    plt.show()

    print("Observação: Em geral, espera-se uma distribuição aproximadamente unimodal, concentrada próximo à tensão nominal (em torno de ~240 V), com alguma dispersão.")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 8- Consumo médio por mês em todo o período.

if Path(DATA_FILE).exists():
    df['YearMonth'] = df['Date_dt'].dt.to_period('M')
    monthly_mean = df.groupby('YearMonth')['Global_active_power'].mean().to_timestamp()
    display(monthly_mean.head())
else:
    print("Arquivo não encontrado.")

In [ ]:
# 9- Dia com maior consumo de energia ativa global (média diária).

if Path(DATA_FILE).exists():
    daily_mean = df.groupby('Date_dt')['Global_active_power'].mean()
    max_day = daily_mean.idxmax()
    print("Dia com maior consumo médio de Global_active_power:", max_day.date(), "—", daily_mean.max(), "kW")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 10- Consumo médio: dias de semana vs finais de semana.

if Path(DATA_FILE).exists():
    df['is_weekend'] = df['Date_dt'].dt.weekday >= 5
    daily = df.groupby(['Date_dt','is_weekend'])['Global_active_power'].mean().reset_index()
    weekday_mean = daily.loc[~daily['is_weekend'],'Global_active_power'].mean()
    weekend_mean = daily.loc[daily['is_weekend'],'Global_active_power'].mean()
    print(f"Média (dias de semana): {weekday_mean:.4f} kW")
    print(f"Média (finais de semana): {weekend_mean:.4f} kW")
else:
    print("Arquivo não encontrado.")


In [ ]:
# 11- Correlação entre GA, GR, Voltage e GI.

if Path(DATA_FILE).exists():
    corr_cols = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity']
    corr = df[corr_cols].corr()
    display(corr)
else:
    print("Arquivo não encontrado.")

In [ ]:
# 12- Nova variável Total_Sub_metering.

if Path(DATA_FILE).exists():
    df['Total_Sub_metering'] = df[['Sub_metering_1','Sub_metering_2','Sub_metering_3']].sum(axis=1, skipna=True)
    display(df[['Sub_metering_1','Sub_metering_2','Sub_metering_3','Total_Sub_metering']].head())
else:
    print("Arquivo não encontrado.")

In [ ]:
# 13- Verificar mês em que Total_Sub_metering ultrapassa a média de Global_active_power.

if Path(DATA_FILE).exists():
    monthly = df.groupby('YearMonth').agg({
        'Total_Sub_metering':'mean',
        'Global_active_power':'mean'
    })
    cond = monthly['Total_Sub_metering'] > monthly['Global_active_power']
    result = monthly[cond]
    print("Meses onde a média de Total_Sub_metering > média de Global_active_power:")
    display(result)
else:
    print("Arquivo não encontrado.")

In [ ]:
# 14- Série temporal de Voltage para 2008.

if Path(DATA_FILE).exists():
    mask_2008 = df['Date_dt'].dt.year == 2008
    df_2008 = df.loc[mask_2008].copy()
    df_2008 = df_2008.sort_values('DateTime')
    plt.figure()
    plt.plot(df_2008['DateTime'], df_2008['Voltage'])
    plt.title("Voltage — Série Temporal (2008)")
    plt.xlabel("Data")
    plt.ylabel("Voltage (V)")
    plt.show()
else:
    print("Arquivo não encontrado.")

In [ ]:
# 15- Comparar consumo entre meses de verão e inverno (Hemisfério Norte).

if Path(DATA_FILE).exists():
    # Verão (HN): Jun, Jul, Ago; Inverno (HN): Dez, Jan, Fev
    df['month'] = df['Date_dt'].dt.month
    summer_months = [6,7,8]
    winter_months = [12,1,2]
    summer = df[df['month'].isin(summer_months)].groupby('Date_dt')['Global_active_power'].mean().mean()
    winter = df[df['month'].isin(winter_months)].groupby('Date_dt')['Global_active_power'].mean().mean()
    print(f"Média verão (HN): {summer:.4f} kW")
    print(f"Média inverno (HN): {winter:.4f} kW")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 16- Amostragem aleatória de 1% e comparação da distribuição de Global_active_power.

if Path(DATA_FILE).exists():
    full = df['Global_active_power'].dropna()
    sample = full.sample(frac=0.01, random_state=42)
    plt.figure()
    full.hist(bins=60, alpha=0.6)
    plt.title("Distribuição — Base Completa (Global_active_power)")
    plt.xlabel("kW")
    plt.ylabel("Frequência")
    plt.show()

    plt.figure()
    sample.hist(bins=60, alpha=0.6)
    plt.title("Distribuição — Amostra 1% (Global_active_power)")
    plt.xlabel("kW")
    plt.ylabel("Frequência")
    plt.show()

    print("Compare visualmente as distribuições para similaridade geral de formato/caudas.")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 17- Normalização (Min-Max Scaling) das variáveis numéricas principais.

if Path(DATA_FILE).exists():
    num_cols = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity',
                'Sub_metering_1','Sub_metering_2','Sub_metering_3','Total_Sub_metering']
    data_num = df[num_cols].dropna()
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(data_num)
    scaled_df = pd.DataFrame(scaled, columns=num_cols, index=data_num.index)
    display(scaled_df.head())
else:
    print("Arquivo não encontrado.")

In [ ]:
# 18- K-means para segmentar dias (3 clusters) e interpretação.

if Path(DATA_FILE).exists():
    # Featurização diária
    feats = df.groupby('Date_dt').agg({
        'Global_active_power':'mean',
        'Global_reactive_power':'mean',
        'Voltage':'mean',
        'Global_intensity':'mean',
        'Total_Sub_metering':'mean'
    }).dropna()

    scaler_days = MinMaxScaler()
    X = scaler_days.fit_transform(feats)

    km = KMeans(n_clusters=3, random_state=42, n_init=10)
    feats['cluster'] = km.fit_predict(X)

    print("Tamanho dos clusters:")
    print(feats['cluster'].value_counts().sort_index())

    print("\nMédias por cluster:")
    display(feats.groupby('cluster').mean())

    # Interpretação (texto)
    print("\nInterpretação geral: clusters com maior média de Global_active_power/Total_Sub_metering tendem a representar dias de maior consumo; tensões/reativa podem distinguir perfis elétricos distintos.")
else:
    print("Arquivo não encontrado.")


In [ ]:
# 19- Decomposição de série temporal (6 meses) para Global_active_power.

if Path(DATA_FILE).exists():
    # Série por hora para 6 meses iniciando em 2007-01 (ajuste se desejar)
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
    ts_hourly = df.set_index('DateTime')['Global_active_power'].resample('1H').mean().dropna()

    # Seleciona janela de 6 meses
    start = pd.Timestamp('2007-01-01')
    end = start + pd.DateOffset(months=6)
    ts_6m = ts_hourly.loc[start:end].dropna()

    result = seasonal_decompose(ts_6m, model='additive', period=24)  # sazonalidade diária (24h)
    # Plot manual em figuras separadas para cumprir a regra (1 por figura)
    plt.figure()
    plt.plot(result.observed.index, result.observed.values)
    plt.title("Decomposição — Observado (6 meses)")
    plt.show()

    plt.figure()
    plt.plot(result.trend.index, result.trend.values)
    plt.title("Decomposição — Tendência")
    plt.show()

    plt.figure()
    plt.plot(result.seasonal.index, result.seasonal.values)
    plt.title("Decomposição — Sazonalidade (≈24h)")
    plt.show()

    plt.figure()
    plt.plot(result.resid.index, result.resid.values)
    plt.title("Decomposição — Resíduo")
    plt.show()
else:
    print("Arquivo não encontrado.")

In [ ]:
# 20- Regressão linear simples para prever Global_active_power a partir de Global_intensity.

if Path(DATA_FILE).exists():
    data = df[['Global_active_power','Global_intensity']].dropna()
    X = data[['Global_intensity']].values
    y = data['Global_active_power'].values
    # Treino/teste simples (holdout)
    n = len(data)
    split = int(n*0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    # Calculate RMSE by taking the square root of the mean squared error
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print("Coeficiente (slope):", lr.coef_[0])
    print("Intercepto:", lr.intercept_)
    print("RMSE (teste):", rmse)
else:
    print("Arquivo não encontrado.")

In [ ]:
# 21- Séries temporais por hora — índice datetime, reamostragem 1h e horários de maior consumo.


if Path(DATA_FILE).exists():
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
    ts = df.set_index('DateTime')['Global_active_power'].resample('1H').mean()
    display(ts.head())

    # Horários de maior consumo médio ao longo do dia (0–23)
    hourly_mean = df.set_index('DateTime')['Global_active_power'].groupby(df['DateTime'].dt.hour).mean()
    print("Médias por hora (0–23):")
    display(hourly_mean.sort_values(ascending=False))
else:
    print("Arquivo não encontrado.")

In [ ]:
# 22- Autocorrelação do consumo (lags 1h, 24h, 48h).

if Path(DATA_FILE).exists():
    ts = df.set_index('DateTime')['Global_active_power'].resample('1H').mean().dropna()
    acf_1 = ts.autocorr(lag=1)
    acf_24 = ts.autocorr(lag=24)
    acf_48 = ts.autocorr(lag=48)
    print(f"Autocorrelação 1h:  {acf_1:.4f}")
    print(f"Autocorrelação 24h: {acf_24:.4f}")
    print(f"Autocorrelação 48h: {acf_48:.4f}")
    print("Padrões diários existem quando a autocorrelação em 24h é significativamente positiva.")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 23- PCA (2 componentes) nas variáveis GA, GR, Voltage e GI.

if Path(DATA_FILE).exists():
    cols = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity']
    data_pca = df[cols].dropna()
    scaler_pca = MinMaxScaler()
    Xp = scaler_pca.fit_transform(data_pca)

    pca = PCA(n_components=2, random_state=42)
    pca_scores = pca.fit_transform(Xp)
    print("Variância explicada por componente:", pca.explained_variance_ratio_)
else:
    print("Arquivo não encontrado.")

In [ ]:
# 24- Visualização de clusters no espaço do PCA (3 clusters).

if Path(DATA_FILE).exists():
    cols = ['Global_active_power','Global_reactive_power','Voltage','Global_intensity']
    data_pca = df[cols].dropna()
    scaler_pca = MinMaxScaler()
    Xp = scaler_pca.fit_transform(data_pca)
    pca = PCA(n_components=2, random_state=42)
    Z = pca.fit_transform(Xp)

    km2 = KMeans(n_clusters=3, random_state=42, n_init=10)
    labels = km2.fit_predict(Z)

    plt.figure()
    plt.scatter(Z[:,0], Z[:,1], s=5, alpha=0.5, c=labels)
    plt.title("PCA (2D) com K-Means (k=3)")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()

    print("Pergunta: os grupos se separam de forma clara? Observe a sobreposição dos pontos no plano.")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 25 Regressão Linear vs Polinomial (grau 2) — Global_active_power ~ Voltage.

if Path(DATA_FILE).exists():
    data = df[['Global_active_power','Voltage']].dropna()
    X = data[['Voltage']].values
    y = data['Global_active_power'].values

    n = len(data)
    split = int(n*0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Linear
    lr2 = LinearRegression()
    lr2.fit(X_train, y_train)
    y_pred_lin = lr2.predict(X_test)
    rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))

    # Polinomial grau 2
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    lr_poly = LinearRegression()
    lr_poly.fit(X_train_poly, y_train)
    y_pred_poly = lr_poly.predict(X_test_poly)
    rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))

    print(f"RMSE Linear:     {rmse_lin:.5f}")
    print(f"RMSE Polinomial: {rmse_poly:.5f}")

    # Curvas ajustadas (amostra de teste ordenada por Voltage para visual)
    order = np.argsort(X_test[:,0])
    Xo = X_test[order]
    yo_lin = y_pred_lin[order]
    yo_poly = y_pred_poly[order]

    plt.figure()
    plt.scatter(X_test[:,0], y_test, s=5, alpha=0.3)
    plt.plot(Xo[:,0], yo_lin)
    plt.title("Ajuste — Regressão Linear (teste)")
    plt.xlabel("Voltage (V)")
    plt.ylabel("Global_active_power (kW)")
    plt.show()

    plt.figure()
    plt.scatter(X_test[:,0], y_test, s=5, alpha=0.3)
    plt.plot(Xo[:,0], yo_poly)
    plt.title("Ajuste — Regressão Polinomial Grau 2 (teste)")
    plt.xlabel("Voltage (V)")
    plt.ylabel("Global_active_power (kW)")
    plt.show()
else:
    print("Arquivo não encontrado.")

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

plt.rcParams.update({'figure.figsize': (10,4), 'axes.grid': True})

DATA_FILE = "energydata_complete.csv"
if not Path(DATA_FILE).exists():
    print(f"⚠️ Arquivo '{DATA_FILE}' não encontrado em {Path.cwd()}")
else:
    print("✅ Arquivo encontrado.")

In [ ]:
# 26- Carregamento e inspeção inicial

if Path(DATA_FILE).exists():
    df = pd.read_csv(DATA_FILE)
    print('Shape:', df.shape)
    print('\n.info():')
    display(df.info())
    print('\n.describe():')
    display(df.describe().T)
else:
    print("Coloque o arquivo na mesma pasta do notebook e rode novamente.")


In [ ]:
# 27- Distribuição do consumo — Appliances

if Path(DATA_FILE).exists():
    # Histograma
    plt.figure()
    df['Appliances'].hist(bins=50)
    plt.title("Histograma — Appliances")
    plt.xlabel("Appliances (Wh)")
    plt.ylabel("Frequência")
    plt.show()

    # Série temporal (amostra para visualização se for muito grande)
    plt.figure()
    # converter date se existir coluna 'date' e plotar uma janela de 7 dias
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        sample = df.set_index('date')['Appliances'].resample('1H').mean().dropna()
        sample_window = sample.sort_index().iloc[:24*7]  # primeira semana
        plt.plot(sample_window.index, sample_window.values)
        plt.title("Série temporal — Appliances (primeira semana)")
        plt.xlabel("Data")
        plt.ylabel("Appliances (Wh)")
        plt.show()
    else:
        plt.plot(df['Appliances'].values[:24*7])
        plt.title("Série temporal (primeiros 7 dias aproximados) — Appliances")
        plt.xlabel("Índice")
        plt.ylabel("Appliances (Wh)")
        plt.show()

    print("Pergunta: o consumo tende a se concentrar em valores baixos ou altos?")
    print("Observe o histograma e a escala (Wh). Em muitos datasets de consumo doméstico, a massa está em valores baixos com cauda à direita.")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 28- Correlações com variáveis ambientais.

if Path(DATA_FILE).exists():
    # Selecionar variáveis ambientais típicas (ajuste se nomes diferentes)
    candidates = ['T_out','T1','T2','T3','T4','T5','T6','T7','T8','T9',
                  'RH_out','RH_1','RH_2','RH_3','RH_4','RH_5','RH_6','RH_7','RH_8','RH_9']
    present = [c for c in candidates if c in df.columns]
    cols = ['Appliances'] + present
    corr = df[cols].corr()
    display(corr.loc['Appliances'].sort_values(ascending=False))
    print("\nPergunta: quais fatores têm mais relação com o consumo?")
    print("Olhe os maiores valores absolutos de correlação com 'Appliances'.")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 29- Normalização (Min-Max Scaling) das variáveis numéricas.

if Path(DATA_FILE).exists():
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    scaler = MinMaxScaler()
    df_scaled = df.copy()
    df_scaled[num_cols] = scaler.fit_transform(df[num_cols].fillna(0))
    print("Exemplo das colunas numéricas escaladas:")
    display(df_scaled[num_cols].head())
else:
    print("Arquivo não encontrado.")

In [ ]:
# 30- PCA — reduzir para 2 componentes principais.

if Path(DATA_FILE).exists():
    numeric = df.select_dtypes(include=[np.number]).dropna(axis=1, how='all')
    cols_pca = numeric.columns.tolist()
    Xp = MinMaxScaler().fit_transform(numeric.fillna(0))
    pca = PCA(n_components=2, random_state=42)
    Z = pca.fit_transform(Xp)
    print("Explained variance ratio:", pca.explained_variance_ratio_)
    plt.figure()
    plt.scatter(Z[:,0], Z[:,1], s=4, alpha=0.6)
    plt.title("PCA — 2 componentes (todos os registros)")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()
    print("Pergunta: aparecem padrões ou agrupamentos naturais? Observe o gráfico para possíveis regiões de maior densidade.")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 31- Regressão Linear Múltipla — Appliances ~ variáveis ambientais.

if Path(DATA_FILE).exists():
    # Definir features ambientais — escolher as presentes
    env_candidates = ['T_out','RH_out']  # defaults
    # expandir se colunas T1.. existirem: escolha algumas representativas
    for c in ['T1','T2','T3','RH_1','RH_2','RH_3']:
        if c in df.columns:
            env_candidates.append(c)
    features = [c for c in env_candidates if c in df.columns]
    data = df[features + ['Appliances']].dropna()
    X = data[features].values
    y = data['Appliances'].values

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

    lr = LinearRegression()
    lr.fit(X_train,y_train)
    y_pred = lr.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print("Features usadas:", features)
    print("R^2:", r2)
    print("RMSE:", rmse)
else:
    print("Arquivo não encontrado.")

In [ ]:
# 32- Random Forest Regressor — comparar com Regressão Linear.

if Path(DATA_FILE).exists():
    # Reusar X_train, X_test, y_train, y_test do passo anterior (se não existirem, refazer)
    try:
        X_train, X_test, y_train, y_test
    except NameError:
        data = df[features + ['Appliances']].dropna()
        features = [c for c in env_candidates if c in df.columns] # Re-define features if necessary
        X = data[features].values
        y = data['Appliances'].values
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train,y_train)
    y_pred_rf = rf.predict(X_test)
    rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
    print("RMSE Regressão Linear (ref):", rmse)
    print("RMSE Random Forest:", rmse_rf)
else:
    print("Arquivo não encontrado.")

In [ ]:
# 33- K-Means clustering (k=3..5)

if Path(DATA_FILE).exists():
    # Usar apenas variáveis numéricas escaladas para clustering
    num = df.select_dtypes(include=[np.number]).fillna(0)
    num_s = MinMaxScaler().fit_transform(num)
    inertias = {}
    for k in range(3,6):
        km = KMeans(n_clusters=k, random_state=42, n_init=10)
        km.fit(num_s)
        inertias[k] = km.inertia_
        print(f"k={k}, inertia={km.inertia_:.2f}")
    # Aplicar k=3 por exemplo e mostrar centros (desescalados apenas para interpretação relativa)
    km3 = KMeans(n_clusters=3, random_state=42, n_init=10)
    labels = km3.fit_predict(num_s)
    df['k3_cluster'] = labels
    print("Tamanho dos clusters (k=3):", df['k3_cluster'].value_counts().to_dict())
    display(df.groupby('k3_cluster')['Appliances'].describe())
    print("Interprete os perfis por média/mediana de 'Appliances' e outras variáveis.")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 34- Classificação binária — alto vs baixo consumo

if Path(DATA_FILE).exists():
    median = df['Appliances'].median()
    df['high_consumption'] = (df['Appliances'] > median).astype(int)
    print("Mediana Appliances:", median)
    display(df['high_consumption'].value_counts())

    # Selecionar features — usar as mesmas ambientais do modelo de regressão
    data_cls = df[features + ['high_consumption']].dropna()
    Xc = data_cls[features].values
    yc = data_cls['high_consumption'].values
    Xc_train,Xc_test,yc_train,yc_test = train_test_split(Xc,yc,test_size=0.2,random_state=42)
    # Logistic Regression
    log = LogisticRegression(max_iter=1000)
    log.fit(Xc_train,yc_train)
    y_pred_log = log.predict(Xc_test)
    # Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=100, random_state=42)
    rfc.fit(Xc_train,yc_train)
    y_pred_rfc = rfc.predict(Xc_test)

    print("Modelos treinados: Logistic Regression e Random Forest (Classifier)")
else:
    print("Arquivo não encontrado.")

In [ ]:
# 35- Avaliação de classificação

if Path(DATA_FILE).exists():
    print("=== Logistic Regression ===")
    print("Accuracy:", accuracy_score(yc_test, y_pred_log))
    print("Precision:", precision_score(yc_test, y_pred_log))
    print("Recall:", recall_score(yc_test, y_pred_log))
    print("F1:", f1_score(yc_test, y_pred_log))
    print("\nConfusion matrix:")
    print(confusion_matrix(yc_test, y_pred_log))

    print("\n=== Random Forest Classifier ===")
    print("Accuracy:", accuracy_score(yc_test, y_pred_rfc))
    print("Precision:", precision_score(yc_test, y_pred_rfc))
    print("Recall:", recall_score(yc_test, y_pred_rfc))
    print("F1:", f1_score(yc_test, y_pred_rfc))
    print("\nConfusion matrix:")
    print(confusion_matrix(yc_test, y_pred_rfc))

    print("\nPergunta: o modelo erra mais para alto ou para baixo consumo?")
    print("Compare valores de falso positivo (prevê alto quando é baixo) e falso negativo (prevê baixo quando é alto) nas matrizes de confusão.")
else:
    print("Arquivo não encontrado.")
